# imports

In [1]:
!pip -q install evaluate

In [2]:
from datasets import load_dataset, Dataset

import datasets
import torch

import numpy as np
import pandas as pd
from tqdm.auto import tqdm


from transformers import TrainingArguments, Trainer
import evaluate


from accelerate import Accelerator
from accelerate.utils import set_seed 

from transformers import GenerationConfig
from torch.optim import AdamW
# from transformers import get_scheduler
import torch.optim.lr_scheduler as lr_scheduler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

# load data, toeknizer and model

In [3]:
train_df = pd.read_csv('/kaggle/input/pre-processed-data/train_preprocessed.csv')
test_df = pd.read_csv('/kaggle/input/pre-processed-data/test_preprocessed.csv')

In [4]:
train_df.head()

,keyword,location,text,target
0,neutral,us,Our Deeds Reason earthquake May ALLAH Forgive us,1
1,fire,canada,Forest fire near La Ronge Sask Canada,1
2,evacuation,nowhere,All residents asked shelter place notified off...,1
3,evacuation,california,people receive wildfires evacuation orders Cal...,1
4,smoke,nowhere,Just got sent photo Ruby Alaska smoke wildfire...,1


In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [6]:
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

In [7]:
train_ds

Dataset({
    features: ['keyword', 'location', 'text', 'target'],
    num_rows: 7613
})

In [8]:
checkpoint = "facebook/bart-large-cnn"  # accuracy .824988


In [9]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [10]:
id2label = {0: "Not Disaster", 1: "Disaster"}
label2id = {"Not Disaster": 0, "Disaster": 1}

In [11]:
base_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['classification_head.out_proj.bias', 'classification_head.dense.bias', 'classification_head.out_proj.weight', 'classification_head.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
base_model

BartForSequenceClassification(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): Layer

# Preprocessing

In [13]:
def tokenize_example(example):
    return tokenizer(example["text"], example["keyword"], truncation=True)

tokenized_train_data = train_ds.map(tokenize_example, batched=True)
tokenized_test_data = test_ds.map(tokenize_example, batched=True, remove_columns = test_ds.column_names)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [14]:
tokenized_train_data

Dataset({
    features: ['keyword', 'location', 'text', 'target', 'input_ids', 'attention_mask'],
    num_rows: 7613
})

In [15]:
tokenized_train_data = tokenized_train_data.remove_columns(["keyword", "location", "text"])
tokenized_train_data = tokenized_train_data.rename_column("target", "labels")
tokenized_train_data.set_format("torch")
tokenized_test_data.set_format("torch")
tokenized_train_data.features, tokenized_test_data.features

({'labels': Value(dtype='int64', id=None),
  'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
  'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)},
 {'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
  'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)})

In [16]:
train_val_split = tokenized_train_data.train_test_split(0.8, shuffle=True)

In [17]:
BATCH_SIZE = 64

from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_val_split["train"], shuffle=True, batch_size=BATCH_SIZE, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    train_val_split["test"], batch_size=BATCH_SIZE, collate_fn=data_collator
)

test_dataloader = DataLoader(
    tokenized_test_data, batch_size=BATCH_SIZE, collate_fn=data_collator
)

In [18]:
tokenized_train_data

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 7613
})

In [19]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([64]),
 'input_ids': torch.Size([64, 37]),
 'attention_mask': torch.Size([64, 37])}

In [20]:
accuracy = evaluate.load("accuracy")



def compute_metrics(eval_pred):

    predictions, labels = eval_pred

    # Use np.argmax to get the predicted class indices from logits
    predictions = np.argmax(predictions[0], axis=1)
    

    # If the labels are one-hot encoded, convert them to class indices
    if len(labels.shape) > 1:
        labels = np.argmax(labels, axis=-1)

    return accuracy.compute(predictions=predictions, references=labels)

In [21]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=train_val_split["train"],
    eval_dataset=train_val_split["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.422167,0.832540
2,No log,0.471608,0.823182
3,No log,0.792110,0.810868
4,No log,0.809628,0.815630


TrainOutput(global_step=384, training_loss=0.2833700974782308, metrics={'train_runtime': 381.4265, 'train_samples_per_second': 15.961, 'train_steps_per_second': 1.007, 'total_flos': 407560594679904.0, 'train_loss': 0.2833700974782308, 'epoch': 4.0})

In [22]:
import evaluate

metric = evaluate.load( "accuracy")

base_model.eval()
for batch in tqdm(eval_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = base_model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

  0%|          | 0/96 [00:00<?, ?it/s]

{'accuracy': 0.8325398128386143}

In [ ]:
# optimizer = AdamW(base_model.parameters(), lr=2e-05)

# num_epochs = 10
# num_training_steps = num_epochs * len(train_dataloader)

# lr_scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0, total_iters=num_training_steps)

# print(num_training_steps)

# base_model.to(device)


# from tqdm.auto import tqdm

# progress_bar = tqdm(range(num_training_steps))

# base_model.train()
# for epoch in range(num_epochs):
#     for batch in train_dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = base_model(**batch)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.set_postfix(loss=loss.item())
#         progress_bar.update(1)

# Final Test

In [23]:
import numpy as np

pred = []

base_model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = base_model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    pred.append(predictions.reshape(-1).cpu().numpy())

# Convert the list of arrays into a 1D NumPy array
pred = np.concatenate(pred)

In [24]:
y_pred = pred.reshape(-1,1)

In [25]:
ids = pd.read_csv('/kaggle/input/test-id/test_id.csv')['id']

# merge ids and predictions
submission = pd.DataFrame(np.concatenate((ids.values.reshape(-1, 1), y_pred.reshape(-1,1)), axis=1), columns=['id', 'target'])

# make id int   
submission['id'] = submission['id'].astype('int')

# make target 0 and 1
submission['target'] = submission['target'].apply(lambda x: 0 if x < 0.5 else 1)

In [26]:
submission

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [27]:
rm /kaggle/working/submission.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
rm: cannot remove '/kaggle/working/submission.csv': No such file or directory


In [28]:
# Save submission
submission.to_csv('submission.csv', index=False)